### Luigi Talamo  

Language Science and Technology

Saarland University (Germany)

luigi.talamo@uni-saarland.de

Code available from https://github.com/rahonalab/dhw-1/

Check also these scripts: https://github.com/rahonalab/doing_stuff_with_ciepplus which are used for actual papers !

# Experimenting with miniciep+

Welcome to this interactive Jupyter notebook!

Now that you know the story of miniciep+, let's start writing some code to interact with this wonderful parallel corpus.

## Python libraries for working with the CoNNL-U format
There are at least three libraries for working with the CoNLL-U format:

* conllu: https://pypi.org/project/conllu/
* pyconll: https://pyconll.readthedocs.io/en/stable/
* udapi: https://udapi.readthedocs.io/

I used to employ the first two libraries, then Andy told me about udapi.
Udapi is unfortunately poorly documented, but is a very powerful library. With the present tutorial, I hope to contribute a bit to its documentation. If you are looking at this file with Google colab, you don't need to install anything on your machine.
Later, when you will use Udapi locally, you will have to install the library with:

`pip3 install udapi`

or

`conda install udapi`


## First steps
First, let's import the udapi library in our Python script and copy the conllu files on our colab:

In [13]:
import udapi
!wget https://raw.githubusercontent.com/rahonalab/dhw-1/main/dataset/conllu/it_nomerosa.conllu
!wget https://raw.githubusercontent.com/rahonalab/dhw-1/main/dataset/conllu/ro_nomerosa.conllu
!wget https://raw.githubusercontent.com/rahonalab/dhw-1/main/dataset/conllu/es_nomerosa.conllu
!wget https://raw.githubusercontent.com/rahonalab/dhw-1/main/dataset/conllu/pt_nomerosa.conllu
!wget https://raw.githubusercontent.com/rahonalab/dhw-1/main/dataset/conllu/fr_nomerosa.conllu

--2025-11-28 10:19:27--  https://raw.githubusercontent.com/rahonalab/dhw-1/main/dataset/conllu/it_nomerosa.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34959 (34K) [text/plain]
Saving to: ‘it_nomerosa.conllu.1’

it_nomerosa.conllu. 100%[===================>]  34.14K  --.-KB/s    in 0.004s  

2025-11-28 10:19:27 (8.96 MB/s) - ‘it_nomerosa.conllu.1’ saved [34959/34959]

--2025-11-28 10:19:27--  https://raw.githubusercontent.com/rahonalab/dhw-1/main/dataset/conllu/en_nomerosa.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not 

then, you can load a conllufile with the ``pyconll.load_from_file`` function. In the conllu/ folder, you find five connlu files containing the first sentences of Eco's Il nome della Rosa, in the original Italian version and in four Romance translations. Let's load the original Italian version:

In [6]:
nomerosa = udapi.Document("it_nomerosa.conllu")

## Walk through sentences and tokens of a CoNNL-U file
Each udapi object like our ``it_nomerosa`` contains a list of sentences, which are callable through its ``.bundle`` attribute.
In turn, each sentence is a list of tokens, which are callable through its ``.nodes`` attribute.
There are basically two ways of exploring and collecting data from a sentence:
* a linear walk: we walk through the sentence following the linear ordering of the tokens (i.e., a flat list of tokens);
* a tree walk, we traverse the sentence using the tree structure described by the dependency relation (i.e. a nested tree structure of tokens).

With respect to other libraries, the same udapi object allows for both ways of exploring.

### Linear walk
Let's start with the linear walk, printing all tokens from the file:

In [7]:
for sentence in nomerosa.bundles:
    for token in sentence.nodes:
        # Print tokens
        print(token)

<0#1, Umberto>
<0#2, Eco>
<0#3, ->
<0#4, Il>
<0#5, nome>
<0#6, di>
<0#7, la>
<0#8, rosa>
<0#9, Naturalmente>
<0#10, ,>
<0#11, un>
<0#12, manoscritto>
<0#13, Il>
<0#14, 16>
<0#15, agosto>
<0#16, 1968>
<0#17, mi>
<0#18, fu>
<0#19, messo>
<0#20, tra>
<0#21, le>
<0#22, mani>
<0#23, un>
<0#24, libro>
<0#25, dovuto>
<0#26, a>
<0#27, la>
<0#28, penna>
<0#29, di>
<0#30, tale>
<0#31, abate>
<0#32, Vallet>
<0#33, ,>
<0#34, Le>
<0#35, manuscript>
<0#36, de>
<0#37, Dom>
<0#38, Adson>
<0#39, de>
<0#40, Melk>
<0#41, ,>
<0#42, traduit>
<0#43, en>
<0#44, français>
<0#45, d'>
<0#46, après>
<0#47, l'>
<0#48, édition>
<0#49, de>
<0#50, Dom>
<0#51, J.>
<0#52, Mabillon>
<0#53, (>
<0#54, Aux>
<0#55, Presses>
<0#56, de>
<0#57, l'>
<0#58, Abbaye>
<0#59, de>
<0#60, la>
<0#61, Source>
<0#62, ,>
<0#63, Paris>
<0#64, ,>
<0#65, 1842>
<0#66, )>
<0#67, .>
<1#1, Il>
<1#2, libro>
<1#3, ,>
<1#4, corredato>
<1#5, da>
<1#6, indicazioni>
<1#7, storiche>
<1#8, invero>
<1#9, assai>
<1#10, povere>
<1#11, ,>
<1#12, asseriva>


Our library stores each of the UD annotation fields as an attribute of the object, e.g. `token.form` contains the graphemic form of the token.
These fields are: ``ord``, ``form``, ``lemma``, ``upos``, ``xpos``, ``feats``, ``parent``, ``deprel``, ``deps``, ``misc``.
Remember the fields we saw in the slides? We can loop over the CoNNL-U file and print them. For instance, let's print the four fields we have discussed in the slides, plus the ID (position number: ``ord``) and the graphemic form of the token:

In [8]:
for sentence in nomerosa.bundles:
    for token in sentence.nodes:
        # Print tokens
        print("Token no. "+str(token.ord)+" has form "+str(token.form)+" UPOS: "+str(token.upos)+", FEATS: "+str(token.feats)+", DEPREL:"+str(token.deprel)+" and its head is token "+str(token.parent))

Token no. 1 has form Umberto UPOS: PROPN, FEATS: _, DEPREL:obl and its head is token <0#19, messo>
Token no. 2 has form Eco UPOS: PROPN, FEATS: _, DEPREL:flat:name and its head is token <0#1, Umberto>
Token no. 3 has form - UPOS: PUNCT, FEATS: _, DEPREL:punct and its head is token <0#1, Umberto>
Token no. 4 has form Il UPOS: DET, FEATS: Definite=Def|Gender=Masc|Number=Sing|PronType=Art, DEPREL:det and its head is token <0#5, nome>
Token no. 5 has form nome UPOS: NOUN, FEATS: Gender=Masc|Number=Sing, DEPREL:nsubj:pass and its head is token <0#19, messo>
Token no. 6 has form di UPOS: ADP, FEATS: _, DEPREL:case and its head is token <0#8, rosa>
Token no. 7 has form la UPOS: DET, FEATS: Definite=Def|Gender=Fem|Number=Sing|PronType=Art, DEPREL:det and its head is token <0#8, rosa>
Token no. 8 has form rosa UPOS: NOUN, FEATS: Gender=Fem|Number=Sing, DEPREL:nmod and its head is token <0#5, nome>
Token no. 9 has form Naturalmente UPOS: ADV, FEATS: _, DEPREL:advmod and its head is token <0#19, 

We can easily traverse the CoNNL-U file and collect simple statistics like 'How many verbs are there in the corpus?' or more complex ones like 'How many personal pronouns do behave as direct objects?'

In [ ]:
verb = 0
pron_obj = 0
prons = []
for sentence in nomerosa.bundles:
        for token in sentence.nodes:
            if token.upos == 'VERB':
                verb += 1
            if token.upos == "PRON" and token.deprel == "obj" and "Prs" in token.feats["PronType"]:
                pron_obj += 1
                prons.append(token.form)
print("There are "+str(verb)+" verbs."+" There are also "+str(pron_obj)+" personal pronouns behaving as direct objects. They are "+str(prons))

There are 37 verbs. There are also 3 personal pronouns behaving as direct objects. They are ['me', "m'", "l'"]


### Tree walk
To traverse the CoNLL-U file in a hierarchical way, we just have to use the ``children`` statement

In [ ]:
for sentence in nomerosa.bundles:
    for token in sentence.nodes:
        print(str(token.form)+" has the following dependencies "+str(token.children))


Umberto has the following dependencies [Node<0#2, Eco>, Node<0#3, Il>, Node<0#4, nome>, Node<0#5, della>, Node<0#6, rosa>, Node<0#7, UN>, Node<0#8, MANUSCRIT>, Node<0#10, NATURELLEMENT>, Node<0#11, .>]
Eco has the following dependencies []
Il has the following dependencies []
nome has the following dependencies []
della has the following dependencies []
rosa has the following dependencies []
UN has the following dependencies []
MANUSCRIT has the following dependencies []
, has the following dependencies []
NATURELLEMENT has the following dependencies [Node<0#9, ,>]
. has the following dependencies []
Le has the following dependencies []
16 has the following dependencies [Node<1#1, Le>, Node<1#3, août>, Node<1#5, ,>]
août has the following dependencies [Node<1#4, 1968>]
1968 has the following dependencies []
, has the following dependencies []
on has the following dependencies []
me has the following dependencies []
mit has the following dependencies [Node<1#2, 16>, Node<1#6, on>, Node<

Let's write now a simple script to compute the word order patterns of adjectives, relative clauses and determiners across our five languages. First we loop over the folder where the CoNLL-U files are stored and load them as ``udapi`` object:

In [10]:
import glob
for conllufile in sorted(glob.glob("*.conllu")):
    ud_doc = udapi.Document(conllufile)

Then, we traverse the CoNLL-U files in a linear way, and we apply a very simple algorithm:
* check whether the token is a NOUN (we use UPOS);
* if yes, we determine if it's modified by an adjective, a relative clause and a determiner (we use deprel);
* then, determine the word order: if the id of the head is higher than the id of the modifier, then we have a mod-head pattern, otherwise we have a head-mod pattern.
We set a python dictionary, ``results``, to take note of the results.

In [14]:
import glob
import pprint #better printing
results = {}
for conllufile in sorted(glob.glob("*.conllu")):
    results[conllufile] = {
    "adjectives": {
        "h-m": int(0),
        "m-h": int(0)
    },
    "relative clauses": {
        "h-m": int(0),
        "m-h": int(0)
    },
    "determiners": {
        "h-m": int(0),
        "m-h": int(0)
    }
    }
    nomerosa = udapi.Document(conllufile)
    for sentence in nomerosa.bundles:
        for token in sentence.nodes:
            if token.upos == 'NOUN':
                for mod in token.children: #search through modifiers
                    if mod.deprel == "amod": #do we have an adjectival modification?
                        if token.ord < mod.ord:
                            results[conllufile]["adjectives"]["h-m"] += 1
                        else:
                            results[conllufile]["adjectives"]["m-h"] += 1
                    if mod.deprel in ["acl:relcl","acl"]: #do we have a modifier acting as a relative clause?
                        if token.ord < mod.ord:
                            results[conllufile]["relative clauses"]["h-m"] += 1
                        else:
                            results[conllufile]["relative clauses"]["m-h"] += 1
                    if mod.deprel in ["det"]: #do we have determiners (very broad category here, it could be articles, demonstratives, quantifiers,...)?
                        if token.ord < mod.ord:
                            results[conllufile]["determiners"]["h-m"] += 1
                        else:
                            results[conllufile]["determiners"]["m-h"] += 1
pprint.pprint(results)


{'es_nomerosa.conllu': {'adjectives': {'h-m': 16, 'm-h': 8},
                        'determiners': {'h-m': 1, 'm-h': 48},
                        'relative clauses': {'h-m': 5, 'm-h': 0}},
 'fr_nomerosa.conllu': {'adjectives': {'h-m': 12, 'm-h': 11},
                        'determiners': {'h-m': 0, 'm-h': 57},
                        'relative clauses': {'h-m': 10, 'm-h': 0}},
 'it_nomerosa.conllu': {'adjectives': {'h-m': 12, 'm-h': 9},
                        'determiners': {'h-m': 0, 'm-h': 48},
                        'relative clauses': {'h-m': 6, 'm-h': 0}},
 'pt_nomerosa.conllu': {'adjectives': {'h-m': 11, 'm-h': 8},
                        'determiners': {'h-m': 0, 'm-h': 54},
                        'relative clauses': {'h-m': 7, 'm-h': 0}},
 'ro_nomerosa.conllu': {'adjectives': {'h-m': 10, 'm-h': 10},
                        'determiners': {'h-m': 4, 'm-h': 14},
                        'relative clauses': {'h-m': 9, 'm-h': 0}}}
